In [10]:
# import the pf covariate object types
from term import CovariateVariableTerm, TemporalAnomalyTypes
from transforms import TransformTypes, TransformAndAdjust
from variable import MalariaCovariate

In [2]:
# import the enums needed to define some parameters for tiff cube management
from raster_utilities.cubes.cube_constants import CubeLevels, SpatialSummaryTypes, CubeResolutions
from raster_utilities.aggregation.aggregation_values import TemporalAggregationStats, ContinuousAggregationStats
from raster_utilities.io.TiffFile import SingleBandTiffFile, RasterProps

In [13]:
from datetime import date
from dateutil.relativedelta import relativedelta
import numpy as np
from osgeo import gdal

### Checking the term and variable objects

1. read the csv file containing the definition of the terms to a dictionary. Term = which raw variable, what spatial and temporal summary, what temporal lag, and what temporal anomaly

In [4]:
import csv
with open(r'C:\Users\zool1301.NDPH\Documents\Code_General\pf_covariate_generation\data\term_definitions.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    termRows = [row for row in reader]
termConfigs = {row['TermID']:row for row in termRows}

read the csv file containing the definition of the variables to a dictionary. Variable = which two terms (defined as above), what transform and offset is applied to them, including parameters for the transforms where appropriate

In [5]:
with open(r'C:\Users\zool1301.NDPH\Documents\Code_General\pf_covariate_generation\data\variable_definitions.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    variableRows = [row for row in reader]


define some functions to build the objects from the csv rows

In [6]:
def buildTransformsFromVariableRow(variableCSVRowDict):
    v = variableCSVRowDict
    xformType1 = TransformTypes(v['Transform1'])
    xformType2 = TransformTypes(v['Transform2'])
    offset1 = v['Offset1']
    offset2 = v['Offset2']
    mean1 = v['Mean1']
    mean2 = v['Mean2']
    sd1 = v['SD1']
    sd2 = v['SD2']
    lambda1 = v['Lambda1']
    lambda2 = v['Lambda2']
    thresh1 = v['Threshold1']
    thresh2 = v['Threshold2']
    xForm1 = TransformAndAdjust(transformType=xformType1, 
                                adjustmentOffset=offset1, 
                                meanValue=mean1, 
                                stdValue=sd1, 
                                lambdaValue=lambda1,
                                thresholdValue=thresh1)
    xForm2 = TransformAndAdjust(transformType=xformType2, 
                                adjustmentOffset=offset2, 
                                meanValue=mean2, 
                                stdValue=sd2, 
                                lambdaValue=lambda2,
                                thresholdValue=thresh2)
    return (xForm1, xForm2)

def buildTermFromRowAndTransform(termCSVRowDict, xForm):
    term = CovariateVariableTerm(Name=termCSVRowDict['TermID'],
                                MasterFolder=termCSVRowDict['RootFolder'],
                               CubeResolution=termCSVRowDict['Res'],
                               CubeLevel=termCSVRowDict['CubeLevel'],
                               SpatialSummaryType=termCSVRowDict['SpatialSummary'],
                               TemporalSummaryType=termCSVRowDict['TemporalSummary'],
                               TemporalAnomalyType=termCSVRowDict['TemporalAnomaly'],
                               TemporalLagMonths=termCSVRowDict['TemporalLagMonths'],
                               Transform=xForm)
    return term
    
    
def buildVariableFromCSVRow(variableCSVRowDict):
    v = variableCSVRowDict
    termConfig1 = termConfigs[v['Term1']]
    termConfig2 = termConfigs[v['Term2']]
    xforms = buildTransformsFromVariableRow(variableCSVRowDict)
    term1 = buildTermFromRowAndTransform(termConfig1, xforms[0])
    term2 = buildTermFromRowAndTransform(termConfig2, xforms[1])
    var = MalariaCovariate(term1, term2)
    return var


In [7]:
import os
def createVariableOutput(variableRow, dates, outFolder):
    pfVar = buildVariableFromCSVRow(variableRow)
    print(variableRow['VariableID'])
    for d in dates:
        outname = "{0!s}.{1!s}.{2!s}.{3!s}.{4!s}.{5!s}.tif".format(
            "PF_"+variableRow['VariableID'],
            d.year,
            str(d.month).zfill(2),
            "Data", "5km", "Data")
        outPath = os.path.join(outFolder, outname)
        if os.path.exists(outPath):
            print "skipping {0!s} as it exists".format(outPath)
            continue
        res = pfVar.ProduceMonthData(d, latLims=(85,-60), lonLims=(-180,180))
        
        rprop=RasterProps(gt=res[1], proj=res[2], ndv=res[3], 
                          width=res[0].shape[1], height=res[0].shape[0], 
                          res="5km", datatype=gdal.GDT_Float32)
        writer = SingleBandTiffFile(filePath=outPath)
        writer.SetProperties(rprop)
        writer.Save(res[0])
        

### check making a single variable

In [8]:
PF_v5 = buildVariableFromCSVRow(variableRows[4])

#MalariaCovariate(v1term1, v1term2)

#### check it works for a nominal date

In [9]:
res = PF_v5.ProduceMonthData(date(2013,1,1), latLims=(85,-60), lonLims=(-180,180))

Output date is 2013-01-01; requesting cube object to read data for actual date 2013-01-01
Reading data from part of file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MCD43B4_BRDF_Reflectance\EVI\5km\Monthly\EVI.2013.01.mean.5km.mean.tif
Caching data for file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MCD43B4_BRDF_Reflectance\EVI\5km\Synoptic\EVI.Synoptic.Overall.mean.5km.mean.tif
Reading data from part of file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MCD43B4_BRDF_Reflectance\EVI\5km\Synoptic\EVI.Synoptic.Overall.mean.5km.mean.tif
Applying expression abs(((dataArr + adjOffset) - meanValue) / stdValue)
Output date is 2013-01-01; requesting cube object to read data for actual date 2012-12-01
Reading data from part of file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MOD11A2_LST\LST_Day\5km\Synoptic\LST_Day.Synoptic.12.mean.5km.range.tif
Applying expression where(abs(dataArr + adjOffset)<thresholdValue, (1.0/thresholdValue), 1.0 / (

### Check it works for a nominal date for all variables
Effectively, check that we have data for everything we are going to need

Run for all dates in a series

In [16]:
minDate = date(2000,1,1)
maxDate = date(2016,12,1)
dates=[minDate]
oneMonth = relativedelta(months=1)
while minDate <= maxDate:
    dates.append(minDate)
    minDate += oneMonth
    


Or just run for a single date for testing all vars

In [8]:
dates = [date(2012,1,1)]

do the actual run

In [ ]:
mainFolder = r'C:\temp\PF_Covars'

for row in variableRows:
    variableid = "PF_"+row['VariableID']
    outFolder = os.path.join(mainFolder, variableid, "5km", "Monthly")
    createVariableOutput(row, dates, outFolder)

V1
Output date is 2000-01-01; requesting cube object to read data for actual date 2000-01-01
Reading data from part of file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MOD11A2_LST\LST_Diurnal_Difference\5km\Synoptic\LST_DiurnalDiff.Synoptic.Overall.mean.5km.max.tif
Applying expression (dataArr + adj) ** 2
Output date is 2000-01-01; requesting cube object to read data for actual date 2000-01-01
Reading data from part of file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\Other_Global_Covariates\WorldClim_Precipitation_TFA\A2\5km\Synoptic\WorldClim_TFA_Precip_a2.Synoptic.Overall.Data.5km.mean.tif
Applying expression abs(((dataArr + adj) - meanValue) / stdValue)
skipping C:\temp\PF_Covars\PF_V1\5km\Monthly\PF_V1.2000.01.Data.5km.Data.tif as it exists
Output date is 2000-02-01; requesting cube object to read data for actual date 2000-02-01
Reading data from part of file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MOD11A2_LST\LST_Diurnal_Difference\5km\Synoptic\LST

In [22]:
outName = r'C:/temp/PF_Covars/pf_v16_201201_mine.tif'

In [23]:
res = v16.ProduceMonthData(date(2012,1,1), latLims=(85,-60), lonLims=(-180,180))

Output date is 2012-01-01; requesting cube object to read data for actual date 2011-11-01
Reading data from part of file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MOD11A2_LST\LST_Night\5km\Synoptic\LST_Night.Synoptic.11.mean.5km.range.tif
Applying expression (dataArr + adjOffset) ** lambdaValue
Output date is 2012-01-01; requesting cube object to read data for actual date 2011-11-01
Reading data from part of file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MCD43B4_BRDF_Reflectance\TCW\5km\Synoptic\TCW.Synoptic.11.mean.5km.range.tif
Applying expression where(abs(dataArr + adjOffset)<thresholdValue, (1.0/thresholdValue), 1.0 / (dataArr + adjOffset))


In [12]:
# an existing file to get the properties
t = SingleBandTiffFile(r'C:\Temp\PF_Covars\test_date\PF_V9\5km\Monthly\test_PF_V9.2012.01.Data.5km.Data.tif')

In [9]:
v14t1 = v14.Term1.GenerateDataForDate(date(2012,1,1))[0]
v14t2 = v14.Term2.GenerateDataForDate(date(2012,1,1))[0]

Output date is 2012-01-01; requesting cube object to read data for actual date 2012-01-01
Reading data from complete file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MOD11A2_LST\LST_Night\5km\Monthly\LST_Night.2012.01.mean.5km.mean.tif
Caching data for file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MOD11A2_LST\LST_Night\5km\Synoptic\LST_Night.Synoptic.Overall.mean.5km.mean.tif
Reading data from complete file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MOD11A2_LST\LST_Night\5km\Synoptic\LST_Night.Synoptic.Overall.mean.5km.mean.tif
Applying expression where( (((dataArr+adj)<thresh) & ((dataArr+adj)>=0)), (1.0/thresh), where( ((abs(dataArr+adj)<thresh) & ((dataArr+adj)<=0)), (1.0/(-thresh)), 1.0/(dataArr+adj)))
Output date is 2012-01-01; requesting cube object to read data for actual date 2012-01-01
Reading data from complete file \\map-fs1.ndph.ox.ac.uk\map_data\mastergrids\MODIS_Global\MCD12Q1_Annual_Landcover\IGBP_Landcover_Class14\5km\Annual

In [36]:
# a new file, not currently existing
t1 = SingleBandTiffFile(r'C:/temp/PF_Covars/pf_v14_t1_201201_mine.tif')

In [37]:
t1.SetProperties(t._Properties)

In [ ]:
t1.Save(v14t2)